# Upload mangrove extent to GCS and as an asst to GEE

This notebook gets mangrove extent data, uploads to GCS, and creates an earth engine image collection asset.  
These are the steps:  
* We first need to create a single tiff per year of mangrove extent data.
* We then upload the tiffs to GCS.
* We then create an earthengine image collection asset.
* We then upload each image as an asset to GEE.
* Finally make the data also publicly available in Zenodo.

## Setup

In [1]:
import os
from pathlib import Path
import urllib.parse
import geemap
import ee
import tarfile

%run utils.ipynb

Exception: File `'utils.ipynb.py'` not found.

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


### Set Cloud credentials (could be done through and env file)

In [4]:
# WARNING: Don't forget to auth to google cloud platform
# gcloud auth application-default login --no-launch-browser --project=mangrove-atlas-246414

### Set Variables for GCS and GEE

In [1]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets'

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets', f'{BASE_DIR} is not the correct directory'

# variables
data_version = 'v3'
dataset = 'mangrove_canopy_height'

# Set the Google Cloud params
gc_project_id = "mangrove-atlas-246414"
gcs_prefix = "gs://mangrove_atlas"
gcs_bucket = f'mangrove_atlas'

# raw data source
base_source_data_url = f'https://www.dropbox.com/sh/ruk6m9btrkqb73b/AAC8lWSe7nOfDGzrapMR_uqRa?dl=0'

raw_local_folder = Path(f'{BASE_DIR}/raw/{dataset}')
raw_local_folder.mkdir(parents=True, exist_ok=True)

# Image Collection Information
data_year_range = [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]
ee_image_collection = f'projects/global-mangrove-watch/mangrove-properties/{dataset}-{data_version}'

no_data_values = [0]
pyramiding = 'MEAN'


NameError: name 'Path' is not defined

In [123]:
description = f"""
## Methodology

This dataset shows the amount and density of carbon stored in mangrove biomass and soil.
Total values represent the sum of aboveground carbon, and the soil organic carbon values, 
attributed to mangrove forests. Total values are expressed in Megatonnes of carbon dioxide 
equivalents (Mt CO2e), while mapped carbon density values are depicted as metric tonnes of CO2 
equivalents per hectare (t CO2e / ha). Above-ground estimates of mangrove carbon were obtained 
from [Simard et al. 2019](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665).  

The data were derived from remotely-sensed canopy height measurements and region-specific 
allometric models validated using in-situ measurements in field plots across three continents.  
This was converted to mean AGB carbon using the stoichiometric factor of 0.451 
([Simard et al. 2019](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665)).  

Soil Organic Carbon (SOC) estimates of the top meter of mangrove soils based on a methodology developed 
by Sanderman et al. (2018) were overlaid with 2016 mangrove extent maps from Bunting et al. 2018 
to produce a global map of soil organic carbon at a 30 m spatial resolution.  

Aboveground and soil carbon values originally expressed in metric tonnes (megagrams) of carbon per hectare 
were converted to total carbon using the Bunting et al. (2018) mangrove extent for each country 
and converted to MtCO2e using a conversion factor of 3.67 (Howard et al. 2014).


### {', '.join(list(map(str, data_year_range[:-1])))} and {data_year_range[-1]} maps of mangrove biomass   
"""

In [124]:
extent_collection_properties = ImageCollectionProperties(
    name = 'Mangrove Blue Carbon',
    version = data_version,
    creator = "Global Mangrove Watch (GMW): Aberystwyth University/soloEO/Wetlands International/UNEP-WCMC/JAXA/DOB Ecology",
    description = description,
    identifier = "",
    keywords = "Erosion; Coasts; Natural Infrastructure; Biodiversity; Blue Carbon; Forests; Mangroves; Landcover",
    citation = """Bunting, P., Rosenqvist, A., Lucas, R. M., Rebelo, L. M., Hilarides, L., Thomas, N., … Finlayson, 
                C. M. (2018). The global mangrove watch - A new 2010 global baseline of mangrove extent. 
                Remote Sensing, 10(10). https://doi.org/10.3390/rs10101669  

                Giri, C., Ochieng, E., Tieszen, L. L., Zhu, Z., Singh, A., Loveland, T., … Duke, N. (2011). 
                Status and distribution of mangrove forests of the world using earth observation satellite data. 
                Global Ecology and Biogeography, 20(1), 154–159. https://doi.org/10.1111/j.1466-8238.2010.00584.x  

                Sanderman J, Hengl T, Fiske G et al. (2018) A global map of mangrove forest soil carbon at 30 m spatial 
                resolution. Environmental Research Letters 13: 055002. doi: 10.1088/1748-9326/aabe1c  

                Simard, M., T. Fatoyinbo, C. Smetanka, V.H. Rivera-monroy, E. Castaneda-mova, N. Thomas, 
                and T. Van der stocken. (2019) Global Mangrove Distribution, Aboveground Biomass, and Canopy Height. 
                ORNL DAAC, Oak Ridge, Tennessee, USA. https://doi.org/10.3334/ORNLDAAC/1665 """,
    license = "https://creativecommons.org/licenses/by/4.0/",
    url = "https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1665",
    language = "en", 
    altName = "Mangrove Blue Carbon, Version 3.0",
    distribution = "",
    variableMeasured = "Blue carbon",
    units = "1",
    spatialCoverage = "Global tropics",
    temporalCoverage = ','.join(map(str, data_year_range)),
    dataLineage = "Raster data supplied as tilesets per year, each tilset was combined, and added to Google earth engine as multi-temporal ImageCollection."
)

In [125]:
def generate_manifests(year):
    files = [blob.name for blob in list_gcs(bucket_name = gcs_bucket, 
                    dir_path = f'ee_import_data/gmw_{data_version}_{year}', file_pattern = '*.tif') if blob.name.endswith('.tif')]
    manifest = GEEManifest(
            path = Path(f'{BASE_DIR}/processed/manifest/gmw_{data_version}_{year}'),
            name = f"projects/earthengine-legacy/assets/{ee_image_collection}/gmw_{data_version}_{year}",
            tilesets = [ Tilesets(
                sources = [Sources(uris = [f"{gcs_prefix}/{file}"]) for file in files],
                    )
                ],
            start_time = f'{year}-01-01T00:00:00Z',
            end_time = f'{year}-12-31T00:00:00Z',
            uri_prefix = f"",
            properties = ImageProperties(
                band_nodata_values = no_data_values[0],
                band_pyramiding_policies = pyramiding,
                band_names = 'extent',
                year = year,
            ),
            pyramiding_policy = pyramiding,
            missing_data = {'values': no_data_values}

        )
    return manifest

In [126]:
def extract_path(path: Path):
    # Extract all the contents of zip file in current directory
    for item in path.iterdir():
        if item.suffix == '.tar.gz':
            with tarfile.open(item.as_posix(), 'r') as conpressObj:
                conpressObj.extractall(path)
            
            item.unlink()

In [127]:
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

In [101]:
files = list(list_gcs(bucket_name = gcs_bucket, dir_path = cgs_parent_folder, file_pattern = '*.zip'))

INFO:root:Searching GMW v3.14 geotiffs/*.zip


[<Blob: mangrove_atlas, GMW v3.14 geotiffs/, 1658502357947930>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_1996_gtiff.zip, 1658502502870899>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2007_gtiff.zip, 1658502517196761>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2008_gtiff.zip, 1658502557420942>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2009_gtiff.zip, 1658502628130467>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2010_gtiff.zip, 1658502634901691>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2015_gtiff.zip, 1658502665713112>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2016_gtiff.zip, 1658502699001779>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2017_gtiff.zip, 1658502728032109>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2018_gtiff.zip, 1658502769412890>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2019_gtiff.zip, 1658502783087871>,
 <Blob: mangrove_atlas, GMW v3.14 geotiffs/gmw_v3_2020_gtiff.zip, 1658502810948153>]

### Get the raw files locally

In [37]:
copy_folder_gcs(f'{gcs_prefix}/{cgs_parent_folder}/*.zip', raw_local_folder)

Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_1996_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_1996_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2008_gtiff.zip...
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2007_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2008_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2009_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2007_gtiff.zip
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2009_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2010_gtiff.zip...
Resuming download for /home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2010_gtiff.zip
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2015_gtiff.zip...
Copying gs://mangrove_atlas/GMW v3.14 geotiffs/gmw_v3_2016_gtiff.zip...
Copying gs://mangrove_atlas/GMW v3.14 geo

### Extract them and reupload to GCS in the ee_import_data folder

In [96]:
extract_path_from_zip(raw_local_folder)

folder_unziped = [x for x in raw_local_folder.iterdir() if x.is_dir()]

for folder in folder_unziped:
    copy_folder_gcs(folder, f'{gcs_prefix}/ee_import_data/')

[PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2008'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2007'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2009'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2015'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2016'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2017'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_1996'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2019'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2010'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2020'),
 PosixPath('/home/jovyan/work/datasets/raw/mangrove_extent/gmw_v3_2018')]

#### Clean up the raw data locally as this is expensive in terms of space.

In [121]:
rm_tree(raw_local_folder)

#### Generate manifests for each year that represent an image in the image collection

In [128]:
list_of_manifests = [generate_manifests(year) for year in data_year_range]

INFO:root:Searching ee_import_data/gmw_v3_1996/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2007/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2008/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2009/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2010/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2015/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2016/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2017/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2018/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2019/*.tif
INFO:root:Searching ee_import_data/gmw_v3_2020/*.tif


#### Create the image collection asset and upload the images to GEE

In [129]:
createImageCollection(ee_asset_path = ee_image_collection,
                        properties = extent_collection_properties, 
                        image_list = list_of_manifests)

INFO:root:Created image collection projects/global-mangrove-watch/land-cover/mangrove_extent-v3
INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_1996


Started upload task with ID: SCA6QNE3CPJNNK4J3SZ2BYDZ


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2007


Started upload task with ID: 6BP27MHNFUOHOPPWLBHGUALZ


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2008


...

Started upload task with ID: PJOVGOMHETHY7TNFDOLAXNTN


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2019


Started upload task with ID: VZWXQMIUX7XPRL2JPRXTHWLH


INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove_extent-v3/gmw_v3_2020


Started upload task with ID: GTNM4IUMVSORB4NCG5H5LX57


If we need to add new years to the image collection, we can do that by adding new manifests to the image collection asset.

In [108]:
addImagesToCollection(ee_image_collection, list_of_manifests[0:1])

INFO:root:Task created
INFO:root:Upload image projects/earthengine-legacy/assets/projects/global-mangrove-watch/land-cover/mangrove-extent-test/gmw_v3_1996


Started upload task with ID: CJXPIRIMZJL6MGZ2US6RD3CH


In [112]:
# Check task status
!earthengine task list --status RUNNING

CJXPIRIMZJL6MGZ2US6RD3CH  Upload        Ingest image: "projects/earthengine-lega..  RUNNING    ---


In [ ]:
# Check ImageCollection properties
!earthengine asset info {ee_asset_path}